# 1: Download the zip file

Done.

# 2: Write a set of methods that takes as input one of these images, and then computes real-numbered features as the return.

In [202]:
import os
import numpy as np
import sys
import cv2
%matplotlib inline
import matplotlib.pyplot as plt

from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average

from skimage.feature import corner_harris, corner_subpix, corner_peaks
from skimage.transform import warp, AffineTransform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage import data, exposure
from skimage.feature import blob_dog, blob_log, blob_doh, daisy
from skimage.filters import gaussian_filter
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from imageio import imread
from skimage.transform import resize
from skimage import feature
from skimage.feature import shape_index
from scipy.misc import imsave
from scipy.signal import fftconvolve, find_peaks_cwt
from scipy.ndimage.interpolation import shift
from skimage import measure

from imutils import contours
import imutils

Features we'll use:

1. Correlation coefficient of R and G channels
2. Correlation coefficient of G and B channels
3. Correlation coefficient of B and R channels
4. Average of R channel colors
5. Average of B channel colors
6. Average of G channel colors
7. Variance of R color channel
8. Variance of B color channel
9. Variance of G color channel
10. Corner detection length
11. Corner detection ratio
12. Length of blobs
13. Number of blobs
14. Number of contours
15. Measure of curvature using spherical caps
16. Image array size

In [203]:
def correlation_coeff(image):
    '''
    Calculate the correlation coefficient between each color channel.
    '''
    axis1 = len(image)
    axis2 = len(image[0])
    if len(image.shape) > 2:
        # Reshape and transpose the image to extract color channels
        colors = np.reshape(image, (axis1*axis2, 3)).T
        reds = colors[0]
        greens = colors[1]
        blues = colors[2]
        RG_cc = np.corrcoef(reds, greens)
        GB_cc = np.corrcoef(greens, blues)
        BR_cc = np.corrcoef(blues, reds)
        return RG_cc[0,1], GB_cc[0,1], BR_cc[0,1]
    else:
        return 0, 0, 0

def average_color(image):
    '''
    Find average color per row
    '''
    axis1 = len(image)
    axis2 = len(image[0])
    if len(image.shape) > 2:
        # Reshape and transpose the image to extract color channels
        colors = np.reshape(image, (axis1*axis2, 3)).T
        reds = colors[0]
        greens = colors[1]
        blues = colors[2]
        R_avg = np.average(reds)
        G_avg = np.average(greens)
        B_avg = np.average(blues)
        return R_avg, G_avg, B_avg
    else:
        return 0, 0, 0
    return average_color

def color_channel_variance(image):
    """
    Calculate the variance of each color channel.
    """
    axis1 = len(image)
    axis2 = len(image[0])
    if len(image.shape) > 2:
        colors = np.reshape(image, (axis1*axis2,3)).T
        reds = colors[0]
        greens = colors[1]
        blues = colors[2]
        # find each average
        R_var = np.var(reds)
        G_var = np.var(greens)
        B_var = np.var(blues)
        return R_var, G_var, B_var 
    else:
        return 0, 0, 0
    
def corner_detection(image):
    '''
    Detects corner points using the Harris corner detector and determines the 
    subpixel position of corners
    '''
    # Convert image to greyscale
    image_gray = rgb2gray(image)
    
    # find the corners using Harris corner detector
    coords = corner_peaks(corner_harris(image_gray), min_distance=5)
    coords_subpix = corner_subpix(image_gray, coords, window_size=13)
    
    # find ratio of the maximum vertical distance and maximum horizontal distance between corners 
    if len(coords) > 1:
        max_vertical = max(coords.T[0])
        min_vertical = min(coords.T[0])
        max_horizontal = max(coords.T[1])
        min_horizontal = min(coords.T[1])
        ratio = (max_vertical - min_vertical)/(max_horizontal - min_horizontal)
        if ratio == np.inf:
            return 0, 0
        else:
            return len(coords), ratio
    elif len(coords) == 1:
        return 1, 1
    elif len(coords) == 0:
        return 0, 0

def blob_size(image):
    '''
    Calculate the average size of blobs
    '''
    # Convert image to greyscale
    image_gray = rgb2gray(image)
    
    # Finds blobs in the given grayscale image using Determinant of Hessian method.
    # This is the fastest approach to finding blobs; best for large blobs
    blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)
    
    # Calculate average size of blobs
    if len(blobs_doh) > 0:
        blob_sizes = blobs_doh.T[2]
        avg_size = np.mean(blob_sizes)
        return len(blobs_doh), avg_size
    elif len(blobs_doh) == 0:
        return 0, 0

def contours(image):
    '''
    Find and extract contours on images. 
    Then calculate the number of contours. 
    '''
    image_gray = rgb2gray(image) # Convert to binary images for better accuracy
    
    # Find contours at a constant value of 0.8
    contours = measure.find_contours(image_gray, 0.8)
    
    return len(contours)

def shape_cap_index(image):
    '''
    The shape index is a single valued measure of local curvature. 
    A shape index of 1 represents ‘spherical caps’, the shape of the spots we 
    want to detect.
    '''
    # Convert image to greyscale
    image_gray = rgb2gray(image)
    
    # We want to detect 'spherical caps', so we threshold the shape index map to
    # find points which are 'spherical caps' (~1)
    target = 1
    delta = 0.05
    s = shape_index(image_gray)
    
    point_y, point_x = np.where(np.abs(s - target) < delta)
    point_z = image[point_y, point_x]
    
    # The shape index map produces the shape, even that of noise.
    # In order to reduce the impact of noise, we apply a Gaussian filter to it,
    # and show the results once in
    
    s_smooth = ndi.gaussian_filter(s, sigma=0.5)

    point_y_s, point_x_s = np.where(np.abs(s_smooth - target) < delta)
    point_z_s = image[point_y_s, point_x_s]
    
    return len(point_z_s)


def image_array_size(image):
    '''
    Find the image array size and total number of pixels
    '''
    # Find number of pixels in each axis
    x_axis = len(image)
    y_axis = len(image[0])
    
    # Calculate total number of pixels
    total = x_axis * y_axis
    return total

# Problem 3

In [204]:
# An array of zeroes with dimensions of the number of images and the number of feature
# extractors, respectively
X = np.zeros((4244,16))

# An empty list that we will populate with our classifications
Y = []

counter = 0
ticker = 0
index = 0
for folder in os.listdir("50_categories/"):
    if folder != ".DS_Store":
        for file in os.listdir("50_categories/"+str(folder)):
            # add classification to Y
            Y.append(str(folder))
            # get image
            head,tail = os.path.split(file)
            path = "50_categories/"+str(folder)+"/"+tail
            image = imread(path) 
            # obtain features
            feature1, feature2, feature3 = correlation_coeff(image)
            feature4, feature5, feature6 = average_color(image)
            feature7, feature8, feature9 = color_channel_variance(image)
            feature10, feature11 = corner_detection(image)
            #feature11 = blob_position_detector(image)
            feature12, feature13 = blob_size(image)
            feature14 = contours(image)
            feature15 = shape_cap_index(image)
            feature16 = image_array_size(image)
            # add features to X
            X[index][0] = feature1
            X[index][1] = feature2
            X[index][2] = feature3
            X[index][3] = feature4
            X[index][4] = feature5
            X[index][5] = feature6
            X[index][6] = feature7
            X[index][7] = feature8
            X[index][8] = feature9
            X[index][9] = feature10
            X[index][10] = feature11
            X[index][11] = feature12
            X[index][12] = feature13
            X[index][13] = feature14
            X[index][14] = feature15
            X[index][15] = feature16
            index+=1
            # print progress
            counter+=1
            ticker+=1
            if ticker == 10:
                ticker = 0
                print(100*index/4244, "%")
                
Y = np.array(Y)

/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: invalid value encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in less
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py:142: RuntimeWarning: invalid value encountered in less


0.235626767200754 %


/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/skimage/feature/corner.py:373: RuntimeWarning: divide by zero encountered in true_divide
  return (2.0 / np.pi) * np.arctan((l2 + l1) / (l2 - l1))


0.471253534401508 %
0.706880301602262 %
0.942507068803016 %
1.17813383600377 %
1.413760603204524 %
1.649387370405278 %
1.885014137606032 %
2.1206409048067862 %
2.35626767200754 %
2.591894439208294 %
2.827521206409048 %
3.063147973609802 %
3.298774740810556 %
3.5344015080113103 %
3.770028275212064 %
4.005655042412818 %
4.2412818096135725 %
4.476908576814326 %
4.71253534401508 %
4.948162111215834 %
5.183788878416588 %
5.419415645617343 %
5.655042412818096 %
5.89066918001885 %
6.126295947219604 %
6.361922714420358 %
6.597549481621112 %
6.833176248821866 %
7.0688030160226205 %
7.304429783223374 %
7.540056550424128 %
7.775683317624882 %
8.011310084825636 %
8.24693685202639 %
8.482563619227145 %
8.718190386427898 %
8.953817153628652 %
9.189443920829406 %
9.42507068803016 %
9.660697455230915 %
9.896324222431668 %
10.131950989632422 %
10.367577756833176 %
10.60320452403393 %
10.838831291234685 %
11.074458058435438 %
11.310084825636192 %
11.545711592836946 %
11.7813383600377 %
12.01696512723845

/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in long_scalars


13.195098963242224 %
13.430725730442978 %
13.666352497643732 %
13.901979264844487 %
14.137606032045241 %
14.373232799245994 %
14.608859566446748 %
14.844486333647502 %
15.080113100848257 %
15.315739868049011 %
15.551366635249764 %
15.786993402450518 %
16.022620169651272 %
16.258246936852025 %
16.49387370405278 %
16.729500471253534 %
16.96512723845429 %
17.200754005655043 %
17.436380772855795 %
17.67200754005655 %
17.907634307257304 %
18.14326107445806 %
18.378887841658813 %
18.614514608859565 %
18.85014137606032 %
19.085768143261074 %
19.32139491046183 %
19.557021677662583 %
19.792648444863335 %
20.02827521206409 %
20.263901979264844 %
20.4995287464656 %
20.735155513666353 %
20.970782280867105 %
21.20640904806786 %
21.442035815268614 %
21.67766258246937 %
21.913289349670123 %
22.148916116870875 %
22.38454288407163 %
22.620169651272384 %
22.855796418473137 %
23.091423185673893 %
23.327049952874646 %
23.5626767200754 %
23.798303487276154 %
24.033930254476907 %
24.269557021677663 %
24.505

/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 15138816 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 15663104 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 13107200 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3368026112 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" 

36.99340245051838 %
37.22902921771913 %
37.46465598491989 %
37.70028275212064 %
37.93590951932139 %
38.17153628652215 %
38.407163053722904 %
38.64278982092366 %
38.87841658812441 %
39.114043355325165 %
39.34967012252592 %
39.58529688972667 %
39.82092365692743 %
40.05655042412818 %
40.29217719132893 %
40.52780395852969 %
40.763430725730444 %
40.9990574929312 %
41.23468426013195 %
41.470311027332706 %
41.70593779453346 %
41.94156456173421 %
42.17719132893497 %
42.41281809613572 %
42.64844486333647 %
42.88407163053723 %
43.119698397737984 %
43.35532516493874 %
43.59095193213949 %
43.826578699340246 %
44.062205466541 %
44.29783223374175 %
44.53345900094251 %
44.76908576814326 %
45.00471253534401 %
45.24033930254477 %
45.475966069745525 %
45.711592836946274 %
45.94721960414703 %
46.182846371347786 %
46.41847313854854 %
46.65409990574929 %
46.88972667295005 %
47.1253534401508 %
47.36098020735155 %
47.59660697455231 %
47.832233741753065 %
48.067860508953814 %
48.30348727615457 %
48.5391140433

/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131072 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6553600 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
/Users/sarafinanance/anaconda/envs/datascience/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 83886080 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size,

77.04995287464656 %
77.28557964184732 %
77.52120640904806 %
77.75683317624882 %
77.99245994344957 %
78.22808671065033 %
78.46371347785109 %
78.69934024505184 %
78.93496701225259 %
79.17059377945334 %
79.4062205466541 %
79.64184731385485 %
79.87747408105561 %
80.11310084825637 %
80.34872761545712 %
80.58435438265786 %
80.81998114985862 %
81.05560791705938 %
81.29123468426013 %
81.52686145146089 %
81.76248821866164 %
81.9981149858624 %
82.23374175306314 %
82.4693685202639 %
82.70499528746466 %
82.94062205466541 %
83.17624882186617 %
83.41187558906692 %
83.64750235626767 %
83.88312912346842 %
84.11875589066918 %
84.35438265786993 %
84.59000942507069 %
84.82563619227145 %
85.0612629594722 %
85.29688972667294 %
85.5325164938737 %
85.76814326107446 %
86.00377002827521 %
86.23939679547597 %
86.47502356267672 %
86.71065032987748 %
86.94627709707822 %
87.18190386427898 %
87.41753063147974 %
87.65315739868049 %
87.88878416588125 %
88.124410933082 %
88.36003770028275 %
88.5956644674835 %
88.83129

In [205]:
'''
Shuffle the data 
'''

import random

print(Y[:5])
print(X[:5])

combined = list(zip(X, Y))
random.shuffle(combined)

X[:], Y[:] = zip(*combined)

print(Y[:5])
print(X[:5])
print(X[X!=0.])

['gorilla' 'gorilla' 'gorilla' 'gorilla' 'gorilla']
[[  9.77235699e-01   9.75400072e-01   9.34378916e-01   1.49408592e+02
    1.48160234e+02   1.40587114e+02   5.79529736e+03   6.34444895e+03
    6.03614109e+03   4.20000000e+01   6.25827815e-01   3.60000000e+01
    1.19197531e+01   1.53000000e+02   0.00000000e+00   1.11331000e+05
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  9.88975044e-01   9.93627882e-01   9.74595498e-01   6.19314538e+01
    6.81960504e+01   7.28912773e+01   1.67321734e+03   1.69043980e+03
    1.83256736e+03   4.00000000e+00   4.01315789e+00   8.00000000e+00
    5.83333333e+00   9.40000000e+01   1.00000000e+00   1.19000000e+05
    0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  9.73526804e-01   9.67097528e-01   9.66893784e-01   7.58049625e+01
    9.42212500e+01   8.40715250e+01   3.61508836e+03   3.56010937e+03
    3.48639676e+03   8.20000000e+01   1.43799472e+00   3.50000000e+01
    5.97142857e+00   5.40000000e+02   6.00000000e+00   2.40000000e+05


In [206]:
'''
Separate the data into training data and testing data.
We will use 90% of the data as training data
'''

train = int(len(Y)*0.9)
X_train = X[:train]
Y_train = Y[:train]
print("training size: " + str(len(Y_train)))
# testing set
X_test = X[train:]
Y_test = Y[train:]
print("testing size: " + str(len(Y_test)))

training size: 3819
testing size: 425


In [207]:
# Create a classifier: 
from sklearn.ensemble import RandomForestClassifier

# instantiate classifier object
classifier = RandomForestClassifier(n_estimators=50)

# fit the classification model on training set
classifier.fit(X_train, Y_train)

# make predictions for testing set
pred_rf = classifier.predict(X_test) 

In [208]:
# compute zero-one loss / score & confusion matrix
from sklearn import metrics

rf_01 = metrics.zero_one_loss(Y_test, pred_rf) # zero-one loss
rf_01_score = metrics.accuracy_score(Y_test, pred_rf) # zero-one score
rf_confmat = metrics.confusion_matrix(Y_test, pred_rf) # conf mat

print("Zero-One Loss: " + str(rf_01))
print("Zero-One Score: " + str(rf_01_score))
print("Confusion Matrix:")
print("[i, j] is the # of objects truly in group i but predicted to be in group j")
print(rf_confmat)

Zero-One Loss: 0.978823529412
Zero-One Score: 0.0211764705882
Confusion Matrix:
[i, j] is the # of objects truly in group i but predicted to be in group j
[[2 2 2 ..., 3 2 0]
 [3 0 0 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 ..., 
 [2 0 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [209]:
# compute precision and recall
# Note: precision & recall are for 2-class; multi-class returns weighted avg. prec/recall

rf_precision = metrics.precision_score(Y_test, pred_rf,average="weighted") # TP / (TP + FP)
rf_recall = metrics.recall_score(Y_test, pred_rf,average="weighted") # TP / (TP + FN)

print("Avg. Precision: ",rf_precision)
print("Avg. Recall: ", rf_recall)

Avg. Precision:  0.0186660899654
Avg. Recall:  0.0211764705882


In [210]:
'''
As in class, we fit a random forest with n_estimators = 50
Default settings
'''

# Find the best Random Forest classifier

from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')


# explore 3 different forest sizes and 3 choices of mtry
parameters = {'n_estimators':[20,50,100],  'max_features':[8,10,19], 
             'criterion': ['gini','entropy']}
rf_tune = model_selection.GridSearchCV(RandomForestClassifier(), parameters, 
                                   n_jobs = -1, cv = 5,verbose=1)
rf_opt = rf_tune.fit(X_train, Y_train)

print("Best zero-one score: " + str(rf_opt.best_score_) + "\n")
print("Optimal Model:\n" + str(rf_opt.best_estimator_))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.5min finished


Best zero-one score: 0.0793401413983

Optimal Model:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


We got ~8% accuracy! Not great, but not terrible. 

In [211]:
classifier.feature_importances_

array([ 0.06418071,  0.06375888,  0.06170818,  0.06588672,  0.06521994,
        0.06528838,  0.06686933,  0.0634906 ,  0.06363924,  0.06334286,
        0.07153987,  0.05423789,  0.06162393,  0.06653735,  0.02308432,
        0.07959181,  0.        ,  0.        ,  0.        ])

The most important features are corner detection, image array size, and the red color channel variance, respecrively. 

# Problem 4

Below is our general classifier function that uses X and Y arrays from above. 

In [ ]:
'''
We use the arrays 'X' and 'Y' to run our final classifier. 
Those arrays need to be populated before running this last step
'''

def general_classifier(X_train, Y_train, path):
    # instantiate the classifier using the optimal model determined above and fit the model using our training data
    classifier = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=19, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
    classifier.fit(X_train, Y_train)
    # get the validation set of images, extract their features and convert them to arrays
    files = []
    X_validate = []
    image_index = 0
    for image_file in os.listdir(path):
        X_validate.append(np.zeros(19))
        # get image
        image = imread(os.path.join(path,image_file)) 
        # obtain features
        feature1, feature2, feature3 = correlation_coeff(image)
        feature4, feature5, feature6 = average_color(image)
        feature7, feature8, feature9 = color_channel_variance(image)
        feature10, feature11 = corner_detection(image)
        #feature11 = blob_position_detector(image)
        feature12, feature13 = blob_size(image)
        feature14 = contours(image)
        feature15 = shape_cap_index(image)
        feature16 = image_array_size(image)
        # add features to X
        X[index][0] = feature1
        X[index][1] = feature2
        X[index][2] = feature3
        X[index][3] = feature4
        X[index][4] = feature5
        X[index][5] = feature6
        X[index][6] = feature7
        X[index][7] = feature8
        X[index][8] = feature9
        X[index][9] = feature10
        X[index][10] = feature11
        X[index][11] = feature12
        X[index][12] = feature13
        X[index][13] = feature14
        X[index][14] = feature15
        X[index][15] = feature16
        index+=1
    files = np.array(files)
    X_validate = np.array(X_validate)
    # make predictions for validation set
    pred_rf = classifier.predict(X_validate) 
    # add filenames and predictions to a new file
    file = open("validation.txt","w") 
    file.write("filename \t\t predicted_class \n") 
    file.write("---------------------------------------------- \n") 
    for i in range(len(files)):
        file.write(files[i]+' \t\t '+pred_rf[i]+'\n')
    file.close() 
    print('file saved as "validation.txt"')

In [212]:
'''
Use the following path to run the classifier on your validation set. 
Be sure to run the entire notebook to get the training data. 
'''

#path = "/new/directory/path/"
#run_final_classifier(X,Y,path)

'\nUse the following path to run the classifier on your validation set. \nBe sure to run the entire notebook to get the training data. \n'